In [1]:
# Import necessary libraries;
import numpy as np
from numpy import mean
from numpy import std


import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set_style('white')

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


from mpl_toolkits.mplot3d import Axes3D
from numpy import genfromtxt

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.datasets import load_digits, make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.neighbors import DistanceMetric
from sklearn import tree

In [2]:
fireWall = pd.read_csv('log2.csv')

fireWallXData = fireWall[['Source Port', 'Destination Port','NAT Source Port',
                         'NAT Destination Port','Bytes','Bytes Sent','Bytes Received','Packets',
                        'Elapsed Time (sec)','pkts_sent','pkts_received']]
fireWallyData = fireWall[['Action']]

X = fireWallXData.iloc[:,0:10]
y = fireWallyData.iloc[:,0]


print(y.shape)
print(X.shape)

#print(fireWall.shape)

FireWALLData = pd.concat([X, y], axis=1)
print(FireWALLData.shape)

newX = FireWALLData.iloc[:,0:9]
newy = FireWALLData.iloc[:,10]

from sklearn.preprocessing import LabelEncoder
encodeLabel = LabelEncoder()
newy = encodeLabel.fit_transform(newy)
print(newy)


FireWALLData.loc[FireWALLData['Action'] == 0, 'Action'] = 1
FireWALLData.loc[FireWALLData['Action'] == 1, 'Action'] = 0
FireWALLData.loc[FireWALLData['Action'] == 2, 'Action'] = 1
FireWALLData.loc[FireWALLData['Action'] == 3, 'Action'] = 0

newy = FireWALLData.iloc[:,10]

FireWALLData.head()


(65532,)
(65532, 10)
(65532, 11)
[0 0 0 ... 2 2 2]


,Source Port,Destination Port,NAT Source Port,NAT Destination Port,Bytes,Bytes Sent,Bytes Received,Packets,Elapsed Time (sec),pkts_sent,Action
0,57222,53,54587,53,177,94,83,2,30,1,allow
1,56258,3389,56258,3389,4768,1600,3168,19,17,10,allow
2,6881,50321,43265,50321,238,118,120,2,1199,1,allow
3,50553,3389,50553,3389,3327,1438,1889,15,17,8,allow
4,50002,443,45848,443,25358,6778,18580,31,16,13,allow


In [3]:
X_train, X_test, y_train, y_test = train_test_split( newX, newy, test_size=60000, random_state=1738)


# NEAREST NEIGHBORS TRIAL ONE ON FIRE WALL DATA SET

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)    
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', KNeighborsClassifier())])

    search_space = [
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['euclidean']
                 },
                
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['manhattan']
                 },
                
                {
                
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['chebyshev']
                 },
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['minkowski']
                 },
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_1_kNN = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL ONE RESULTS ---------------------------------------------")

Fitting 2 folds for each of 576 candidates, totalling 1152 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:    6.6s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:    8.2s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL ONE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed:   41.5s finished


In [6]:
TRIAL_1_kNN.cv_results_

{'mean_fit_time': array([0.00875711, 0.00950825, 0.00775623, 0.00825703, 0.00825727,
        0.00750685, 0.00750649, 0.00775671, 0.009758  , 0.00800741,
        0.00950837, 0.00800681, 0.01000845, 0.00725663, 0.00850677,
        0.00725615, 0.01100957, 0.00950873, 0.00725627, 0.00750685,
        0.00725639, 0.00775635, 0.00925779, 0.00875735, 0.00800705,
        0.00750697, 0.00825739, 0.00725603, 0.00850749, 0.00750625,
        0.00925756, 0.01201415, 0.00900781, 0.01050889, 0.00700641,
        0.00925779, 0.00700593, 0.00800669, 0.00800705, 0.01025879,
        0.00800681, 0.00725627, 0.00750697, 0.00675583, 0.00700557,
        0.00800705, 0.00875747, 0.00825715, 0.0120101 , 0.01101005,
        0.01126015, 0.01025867, 0.01125944, 0.01125956, 0.01100922,
        0.0140121 , 0.00950813, 0.01050913, 0.01125956, 0.01151013,
        0.01050925, 0.01075935, 0.01000881, 0.01000869, 0.01050937,
        0.01075947, 0.01276064, 0.01000869, 0.01176035, 0.01276124,
        0.01025891, 0.01050925,

In [7]:
print("---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_1_kNN.cv_results_['rank_test_neg_log_loss']
TRIAL_1_kNN.cv_results_['params'][ np.argmin(TRIAL_1_kNN.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'ball_tree',
 'classifier__metric': 'euclidean',
 'classifier__n_neighbors': 97,
 'classifier__weights': 'distance'}

In [8]:
print("---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------")
TRIAL_1_kNN.cv_results_['rank_test_f1_micro']
TRIAL_1_kNN.cv_results_['params'][ np.argmin(TRIAL_1_kNN.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'ball_tree',
 'classifier__metric': 'manhattan',
 'classifier__n_neighbors': 5,
 'classifier__weights': 'distance'}

In [9]:
print("---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------")
TRIAL_1_kNN.cv_results_['rank_test_roc_auc_ovo']
TRIAL_1_kNN.cv_results_['params'][ np.argmin(TRIAL_1_kNN.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'ball_tree',
 'classifier__metric': 'manhattan',
 'classifier__n_neighbors': 83,
 'classifier__weights': 'distance'}

In [10]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_1_kNN.cv_results_['params'][ np.argmin(TRIAL_1_kNN.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 kNN using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_1_kNN.cv_results_['params'][ np.argmin(TRIAL_1_kNN.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 kNN using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_1_kNN.cv_results_['params'][ np.argmin(TRIAL_1_kNN.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 kNN using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 1 kNN using best NEG LOG LOSS hyperparameters :0.9763833333333334
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 kNN using best F1 hyperparameters :0.9918333333333333
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 kNN using best ROC_AUC hyperparameters :0.9806666666666667
------------------------------------------------------------------------------------------------


# NEAREST NEIGHBORS TRIAL TWO ON FIRE WALL DATA SET


In [11]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)    
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', KNeighborsClassifier())])

    search_space = [
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['euclidean']
                 },
                
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['manhattan']
                 },
                
                {
                
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['chebyshev']
                 },
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['minkowski']
                 },
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_2_kNN = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL TWO RESULTS ---------------------------------------------")

Fitting 2 folds for each of 576 candidates, totalling 1152 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL TWO RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed:   40.7s finished


In [12]:
TRIAL_2_kNN.cv_results_

{'mean_fit_time': array([0.00625575, 0.00850797, 0.00825644, 0.00650501, 0.00950766,
        0.00775552, 0.00900781, 0.01050878, 0.00800693, 0.00750637,
        0.00750685, 0.00650513, 0.00750649, 0.00725687, 0.00750637,
        0.01226091, 0.00800669, 0.00750625, 0.00700641, 0.00725603,
        0.00800693, 0.00850737, 0.00700593, 0.01226068, 0.00700593,
        0.00675535, 0.00825667, 0.00725615, 0.00800681, 0.01100981,
        0.00700593, 0.00775671, 0.00925779, 0.00750649, 0.00825727,
        0.01000917, 0.00775695, 0.00800705, 0.00750625, 0.00750589,
        0.00800705, 0.00800705, 0.00925803, 0.00750613, 0.00675595,
        0.00825727, 0.00750673, 0.00775683, 0.01075912, 0.00950825,
        0.01025879, 0.01025891, 0.01075912, 0.00950837, 0.01000845,
        0.01000845, 0.00975835, 0.01000834, 0.01025856, 0.01125956,
        0.00975883, 0.01050901, 0.01050854, 0.01025879, 0.01050889,
        0.00975859, 0.01025856, 0.00975847, 0.01000857, 0.0122602 ,
        0.01250982, 0.00950825,

In [13]:
print("---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_2_kNN.cv_results_['rank_test_neg_log_loss']
TRIAL_2_kNN.cv_results_['params'][ np.argmin(TRIAL_2_kNN.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'brute',
 'classifier__metric': 'euclidean',
 'classifier__n_neighbors': 89,
 'classifier__weights': 'distance'}

In [14]:
print("---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------")
TRIAL_2_kNN.cv_results_['rank_test_f1_micro']
TRIAL_2_kNN.cv_results_['params'][ np.argmin(TRIAL_2_kNN.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'ball_tree',
 'classifier__metric': 'manhattan',
 'classifier__n_neighbors': 5,
 'classifier__weights': 'distance'}

In [15]:
print("---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------")
TRIAL_2_kNN.cv_results_['rank_test_roc_auc_ovo']
TRIAL_2_kNN.cv_results_['params'][ np.argmin(TRIAL_2_kNN.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'kd_tree',
 'classifier__metric': 'chebyshev',
 'classifier__n_neighbors': 99,
 'classifier__weights': 'distance'}

In [16]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_2_kNN.cv_results_['params'][ np.argmin(TRIAL_2_kNN.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 2 kNN using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_2_kNN.cv_results_['params'][ np.argmin(TRIAL_2_kNN.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 2 kNN using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_2_kNN.cv_results_['params'][ np.argmin(TRIAL_2_kNN.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 2 kNN using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 2 kNN using best NEG LOG LOSS hyperparameters :0.9770333333333333
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 kNN using best F1 hyperparameters :0.9918333333333333
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 kNN using best ROC_AUC hyperparameters :0.9743833333333334
------------------------------------------------------------------------------------------------


# NEAREST NEIGHBORS TRIAL THREE ON FIRE WALL DATA SET


In [17]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)    
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', KNeighborsClassifier())])

    search_space = [
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['euclidean']
                 },
                
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['manhattan']
                 },
                
                {
                
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['chebyshev']
                 },
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['minkowski']
                 },
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_3_kNN = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL THREE RESULTS ---------------------------------------------")

Fitting 2 folds for each of 576 candidates, totalling 1152 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL THREE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed:   38.0s finished


In [18]:
TRIAL_3_kNN.cv_results_

{'mean_fit_time': array([0.00475395, 0.00450468, 0.00575507, 0.00625563, 0.00700593,
        0.00650513, 0.00675535, 0.00700581, 0.00850689, 0.00725579,
        0.00725591, 0.00675583, 0.00700593, 0.00900757, 0.00725627,
        0.00650585, 0.00775611, 0.00700629, 0.00725603, 0.00725627,
        0.00725675, 0.00725627, 0.00725603, 0.00725639, 0.00700617,
        0.00700605, 0.00700569, 0.00700617, 0.00700605, 0.00675595,
        0.00725615, 0.00775623, 0.00700557, 0.00950801, 0.00725651,
        0.00675571, 0.00700617, 0.00700617, 0.00750625, 0.00700605,
        0.00700593, 0.00700569, 0.00650537, 0.00750637, 0.00725567,
        0.00775671, 0.00700605, 0.00675571, 0.01025891, 0.01000881,
        0.01050901, 0.00975847, 0.00950789, 0.01050889, 0.01050854,
        0.0122602 , 0.00975847, 0.00975859, 0.00950789, 0.01025867,
        0.00950813, 0.01025879, 0.01050925, 0.00975859, 0.01025867,
        0.01100945, 0.01000845, 0.01050901, 0.00925767, 0.01025891,
        0.01000845, 0.01000857,

In [19]:
print("---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_3_kNN.cv_results_['rank_test_neg_log_loss']
TRIAL_3_kNN.cv_results_['params'][ np.argmin(TRIAL_3_kNN.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'brute',
 'classifier__metric': 'manhattan',
 'classifier__n_neighbors': 99,
 'classifier__weights': 'distance'}

In [20]:
print("---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------")
TRIAL_3_kNN.cv_results_['rank_test_f1_micro']
TRIAL_3_kNN.cv_results_['params'][ np.argmin(TRIAL_3_kNN.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'ball_tree',
 'classifier__metric': 'manhattan',
 'classifier__n_neighbors': 11,
 'classifier__weights': 'distance'}

In [21]:
print("---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------")
TRIAL_3_kNN.cv_results_['rank_test_roc_auc_ovo']
TRIAL_3_kNN.cv_results_['params'][ np.argmin(TRIAL_3_kNN.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'ball_tree',
 'classifier__metric': 'manhattan',
 'classifier__n_neighbors': 59,
 'classifier__weights': 'distance'}

In [22]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_3_kNN.cv_results_['params'][ np.argmin(TRIAL_3_kNN.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 3 knn using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_3_kNN.cv_results_['params'][ np.argmin(TRIAL_3_kNN.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 3 knn using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_3_kNN.cv_results_['params'][ np.argmin(TRIAL_3_kNN.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 3 knn using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 3 knn using best NEG LOG LOSS hyperparameters :0.9788833333333333
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 knn using best F1 hyperparameters :0.9899666666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 knn using best ROC_AUC hyperparameters :0.9827333333333333
------------------------------------------------------------------------------------------------


# NEAREST NEIGHBORS TRIAL FOUR ON FIRE WALL DATA SET


In [23]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)    
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', KNeighborsClassifier())])

    search_space = [
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['euclidean']
                 },
                
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['manhattan']
                 },
                
                {
                
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['chebyshev']
                 },
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['minkowski']
                 },
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_4_kNN = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FOUR RESULTS ---------------------------------------------")

Fitting 2 folds for each of 576 candidates, totalling 1152 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1942s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done 100 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Batch computation too slow (2.1629s.) Setting batch_size=1.
[Parallel(n_jobs=-1)]: Done 228 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 265 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 292 tasks      | elapsed:   11.0s
[Parallel(n_j

---------------------------------TRIAL FOUR RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed:   37.4s finished


In [24]:
TRIAL_4_kNN.cv_results_

{'mean_fit_time': array([0.00550628, 0.00450683, 0.0057559 , 0.0062542 , 0.00700307,
        0.00650442, 0.00675583, 0.00650573, 0.00650561, 0.00850713,
        0.00700557, 0.00650537, 0.00850725, 0.00700605, 0.00650525,
        0.00675583, 0.00625503, 0.00650609, 0.00725615, 0.00675547,
        0.00675583, 0.00650585, 0.00725615, 0.00650561, 0.00700617,
        0.00675595, 0.00650549, 0.00650561, 0.00700593, 0.00700617,
        0.00600493, 0.00650549, 0.00650537, 0.00850725, 0.00825691,
        0.00675595, 0.00675571, 0.00725627, 0.00725639, 0.00650561,
        0.00700605, 0.00750625, 0.00650573, 0.00700569, 0.00675559,
        0.00725627, 0.00675726, 0.00700617, 0.00950837, 0.00900817,
        0.00925803, 0.00950813, 0.01000905, 0.01050913, 0.01075923,
        0.00950766, 0.00900781, 0.01826572, 0.00950849, 0.0100081 ,
        0.00850725, 0.00975847, 0.00975823, 0.00872934, 0.01050913,
        0.01000857, 0.00975835, 0.00925803, 0.00925791, 0.00925827,
        0.00900781, 0.01025867,

In [25]:
print("---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_4_kNN.cv_results_['rank_test_neg_log_loss']
TRIAL_4_kNN.cv_results_['params'][ np.argmin(TRIAL_4_kNN.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'ball_tree',
 'classifier__metric': 'manhattan',
 'classifier__n_neighbors': 97,
 'classifier__weights': 'distance'}

In [26]:
print("---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------")
TRIAL_4_kNN.cv_results_['rank_test_f1_micro']
TRIAL_4_kNN.cv_results_['params'][ np.argmin(TRIAL_4_kNN.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'brute',
 'classifier__metric': 'euclidean',
 'classifier__n_neighbors': 5,
 'classifier__weights': 'distance'}

In [27]:
print("---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------")
TRIAL_4_kNN.cv_results_['rank_test_roc_auc_ovo']
TRIAL_4_kNN.cv_results_['params'][ np.argmin(TRIAL_4_kNN.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'brute',
 'classifier__metric': 'chebyshev',
 'classifier__n_neighbors': 37,
 'classifier__weights': 'distance'}

In [28]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_4_kNN.cv_results_['params'][ np.argmin(TRIAL_4_kNN.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 4 knn using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_4_kNN.cv_results_['params'][ np.argmin(TRIAL_4_kNN.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 4 knn using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_4_kNN.cv_results_['params'][ np.argmin(TRIAL_4_kNN.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 4 knn using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 4 knn using best NEG LOG LOSS hyperparameters :0.9791
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 knn using best F1 hyperparameters :0.9914666666666667
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 knn using best ROC_AUC hyperparameters :0.9834
------------------------------------------------------------------------------------------------


# NEAREST NEIGHBORS TRIAL FIVE ON FIRE WALL DATA SET


In [29]:
ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = FireWALLData.sample(n = 5000, replace = True)    
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,0:9].values
    ySet = random5000DataPoints.iloc[:,10].values
    ySet = encodeLabel.fit_transform(ySet)
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', KNeighborsClassifier())])

    search_space = [
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['euclidean']
                 },
                
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['manhattan']
                 },
                
                {
                
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['chebyshev']
                 },
                {
                 
                 'classifier': [KNeighborsClassifier()],
                 'classifier__n_neighbors': [5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43,
                                             47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 99],
                 'classifier__weights': ['uniform','distance'],
                 'classifier__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                 'classifier__metric': ['minkowski']
                 },
                
                ]
    
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=2), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_5_kNN = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FIVE RESULTS ---------------------------------------------")

Fitting 2 folds for each of 576 candidates, totalling 1152 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FIVE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 1152 out of 1152 | elapsed:   37.1s finished


In [30]:
TRIAL_5_kNN.cv_results_

{'mean_fit_time': array([0.00550544, 0.00625765, 0.00650549, 0.00800622, 0.00750589,
        0.00700569, 0.00675535, 0.00725627, 0.0090071 , 0.00800645,
        0.00750589, 0.00725627, 0.00700581, 0.00750637, 0.00650513,
        0.00725627, 0.00675523, 0.00700581, 0.00800669, 0.00750625,
        0.00700593, 0.00700605, 0.01526308, 0.00725627, 0.00925815,
        0.00700593, 0.00825727, 0.00800633, 0.00700569, 0.00725555,
        0.00750637, 0.00700605, 0.00750637, 0.00650525, 0.00800645,
        0.00750673, 0.01151025, 0.00800657, 0.00775683, 0.00875747,
        0.00725603, 0.00700569, 0.00825751, 0.00700593, 0.00700593,
        0.00725651, 0.00700617, 0.00700581, 0.00950742, 0.00975811,
        0.00975883, 0.00975871, 0.00975823, 0.00950801, 0.00925815,
        0.00925815, 0.01000857, 0.01226044, 0.01000857, 0.01025927,
        0.00925791, 0.00975871, 0.01050878, 0.01126003, 0.01125932,
        0.01000834, 0.01050878, 0.01075923, 0.01075947, 0.01000845,
        0.01025891, 0.01050913,

In [31]:
print("---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_5_kNN.cv_results_['rank_test_neg_log_loss']
TRIAL_5_kNN.cv_results_['params'][ np.argmin(TRIAL_5_kNN.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'brute',
 'classifier__metric': 'euclidean',
 'classifier__n_neighbors': 97,
 'classifier__weights': 'distance'}

In [32]:
print("---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------")
TRIAL_5_kNN.cv_results_['rank_test_f1_micro']
TRIAL_5_kNN.cv_results_['params'][ np.argmin(TRIAL_5_kNN.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'ball_tree',
 'classifier__metric': 'manhattan',
 'classifier__n_neighbors': 5,
 'classifier__weights': 'distance'}

In [33]:
print("---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------")
TRIAL_5_kNN.cv_results_['rank_test_roc_auc_ovo']
TRIAL_5_kNN.cv_results_['params'][ np.argmin(TRIAL_5_kNN.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------


{'classifier': KNeighborsClassifier(),
 'classifier__algorithm': 'ball_tree',
 'classifier__metric': 'manhattan',
 'classifier__n_neighbors': 89,
 'classifier__weights': 'distance'}

In [34]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_5_kNN.cv_results_['params'][ np.argmin(TRIAL_5_kNN.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 5 KNN using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_5_kNN.cv_results_['params'][ np.argmin(TRIAL_5_kNN.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 5 KNN using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_5_kNN.cv_results_['params'][ np.argmin(TRIAL_5_kNN.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 5 KNN using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 5 KNN using best NEG LOG LOSS hyperparameters :0.9763833333333334
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 KNN using best F1 hyperparameters :0.9918333333333333
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 KNN using best ROC_AUC hyperparameters :0.97995
------------------------------------------------------------------------------------------------
